In [67]:
# BertTopic using a pretrained model
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
import json
import os
from nltk.tokenize import sent_tokenize

! if not nbformat:
!    pip install --upgrade nbformat==4.2.0

fish: Missing end to balance this if statement
 if not nbformat:
 ^^


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [68]:
with open(os.path.join(os.getcwd(), "./../lesswrong_data.json")) as f:
    json_data = json.load(f)["posts"]

preprocessed_corpus = []
preprocessed_tags = []

# using sentence trasnformers for embeddings:
# for post in json_data:
#     # Split into sentences for better context
#     sentences = sent_tokenize(post["text"])
#     post_tags = [post["tags"]] * len(sentences)

#     preprocessed_corpus.extend(sentences)
#     preprocessed_tags.extend(post_tags)

# assert len(preprocessed_corpus) == len(preprocessed_tags), "Number of sentences and tags must match"

# using the entite post as document: 
for post in json_data:
    preprocessed_corpus.append(post["text"])
    preprocessed_tags.append(post["tags"])

print(preprocessed_tags)
print(len(preprocessed_corpus), len(preprocessed_tags))

[['AI'], ['AI'], ['Interpretability (ML & AI)', 'Language Models (LLMs)', 'AI'], ['AI'], ['AI Governance', 'AI Timelines', 'AI'], ['AI Governance', 'Careers', 'AI'], ['OpenAI', 'Recursive Self-Improvement', 'Reinforcement learning', 'Superintelligence', 'AI'], ['AI'], ['AI'], ['Cognitive Science', 'Forecasting & Prediction', 'Utility Functions', 'AI', 'Rationality'], ['AI'], ['Newsletters', 'AI'], ['AI'], ['AI Control', 'AI'], ['AI Alignment Fieldbuilding', 'Philanthropy / Grant making (Topic)', 'AI'], ['Humor', 'Journalism', 'AI'], ['Simulator Theory', 'AI'], ['AI Control', 'DeepMind', 'Economic Consequences of AGI', 'Financial Investing', 'AI'], ['AI Safety Public Materials', 'AI'], ['Newsletters', 'AI'], ['AI Governance', 'AI'], ['Outer Alignment', 'Aligned AI Proposals', 'Gradient Descent', 'Reinforcement learning', 'AI-Assisted Alignment', 'Inner Alignment', 'Value Learning', 'AI'], ['AI'], ['AI'], ['AI Evaluations', 'Introspection', 'Language Models (LLMs)', 'Situational Awarenes

In [69]:
# get unique tags
unique_tags = []
for tags in preprocessed_tags:
    for tag in tags:
        if tag not in unique_tags:
            unique_tags.append(tag)
print(unique_tags)

ctfidf_model = ClassTfidfTransformer(
    seed_words=unique_tags,
    seed_multiplier=5,
    reduce_frequent_words=True
)

['AI', 'Interpretability (ML & AI)', 'Language Models (LLMs)', 'AI Governance', 'AI Timelines', 'Careers', 'OpenAI', 'Recursive Self-Improvement', 'Reinforcement learning', 'Superintelligence', 'Cognitive Science', 'Forecasting & Prediction', 'Utility Functions', 'Rationality', 'Newsletters', 'AI Control', 'AI Alignment Fieldbuilding', 'Philanthropy / Grant making (Topic)', 'Humor', 'Journalism', 'Simulator Theory', 'DeepMind', 'Economic Consequences of AGI', 'Financial Investing', 'AI Safety Public Materials', 'Outer Alignment', 'Aligned AI Proposals', 'Gradient Descent', 'AI-Assisted Alignment', 'Inner Alignment', 'Value Learning', 'AI Evaluations', 'Introspection', 'Situational Awareness', 'AI Alignment Intro Materials', 'Orthogonality Thesis', 'Goal-Directedness', 'Category theory', 'Fixed Point Theorems', 'Formal Proof', 'Gödelian Logic', "Löb's theorem", 'Logic & Mathematics ', 'Logical Uncertainty', 'World Modeling', 'AI Takeoff', 'Consciousness', 'Neuroscience', 'Predictive Pro

In [70]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

topic_model = BERTopic(
    embedding_model=embedding_model,
    min_topic_size=3,
    nr_topics="none",
    ctfidf_model=ctfidf_model,
    verbose=True
)

topics, probs = topic_model.fit_transform(preprocessed_corpus)


2025-06-07 18:30:24,191 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 7/7 [00:09<00:00,  1.33s/it]
2025-06-07 18:30:33,748 - BERTopic - Embedding - Completed ✓
2025-06-07 18:30:33,749 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-07 18:30:34,073 - BERTopic - Dimensionality - Completed ✓
2025-06-07 18:30:34,074 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-07 18:30:34,091 - BERTopic - Cluster - Completed ✓
2025-06-07 18:30:34,092 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-06-07 18:30:34,551 - BERTopic - Representation - Completed ✓
2025-06-07 18:30:34,552 - BERTopic - Topic reduction - Reducing number of topics
2025-06-07 18:30:34,559 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-07 18:30:35,034 - BERTopic - Representation - Completed ✓
2025-06-07 18:30:35,037 - BERTopic - Topic reduction - Reduce

In [71]:
hierarchical_topics = topic_model.hierarchical_topics(preprocessed_corpus)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 11/11 [00:00<00:00, 239.33it/s]


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [tex_font_mjxc_family_src, , ,
                       steering_vector_embeddings_spatial_vectors],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAC6Cw/1h3vqP7oLD/WHe+o/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAFMAAAAAAAAAUwAAAAAAAAC7AAAAAAAAALsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [of_that_is_data_the, , , ai_technical_governance_be_for],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAA2wslvRWPYPzbCyW9FY9g/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACAS8AAAAAAAIBLwAAAAAAAQFDAAAAAAABAUMA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [ai_of_be_and_to, , , opus_claude_anthropic_it_you],
              'type': 'scatter',
              'x': {'bdata': 'NsLJb0Vj2D/x5TBjG5jbP/HlMGMbmNs/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAATsAAAAAAAABOwAAAAAAAwFLAAAAAAADAUsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [material_variables_day_convenience_lunch, , ,
                       and_that_it_to_of],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAACqj6rC9PTjP6qPqsL09OM/8eUwYxuY2z8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACARsAAAAAAAIBGwAAAAAAA4FDAAAAAAADgUMA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [and_that_to_of_it, , , llms_game_delegate_concepts_answer],
              'type': 'scatter',
              'x': {'bdata': 'qo+qwvT04z+tFx79/1HmP60XHv3/UeY/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAgTMAAAAAAACBMwAAAAAAAQFXAAAAAAABAVcA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [agents_village_zochi_their_gpt, , , and_that_to_of_it],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAACJPOBhc37nP4k84GFzfuc/rRce/f9R5j8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACAQcAAAAAAAIBBwAAAAAAAqFHAAAAAAACoUcA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [and_that_to_of_it, , ,
                       hours_estimates_scorecard_costs_compliance],
              'type': 'scatter',
              'x': {'bdata': 'iTzgYXN+5z9EBxXuI+jnP0QHFe4j6Oc/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABoSsAAAAAAAGhKwAAAAAAAwFfAAAAAAADAV8A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [simulator_reward_characters_hacking_simulators, , ,
                       and_of_that_to_it],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAACjKMWVA/7oP6MoxZUD/ug/RAcV7iPo5z8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAOcAAAAAAAAA5wAAAAAAAelLAAAAAAAB6UsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
     

In [64]:
topic_model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'advisor | 01 | principal | alignment | sharkey', 30],
                                   [1, 'data | onionai | long | his | tokens', 25],
                                   [2, 'technical | governance | ai | policy | work', 20],
                                   [3, 'opus | claude | anthropic | it | do', 16],
                                   [4, 'material | variables | day | lunch | convenience', 13],
                                   [5, 'estimates | scorecard | hours | costs | compliance', 13],
                                   [6, 'wishes | wish | all | jobs | ai', 10],
                                   [7, 'elicitation | site | evals | results | ads', 9],
                                   [8, 'steering | vector | embeddings | spatial | vectors', 8],
                                   [9, 'advocacy | political | voting | regulations | bill', 8],
                                   [10, 'tex | font | mjxc | family | src', 8],
                                   [11, 'advocacy | donate | donating | grantmaker | soares', 7],
                                   [12, 'polysemanticity | networks | superposition | claim | solutions',
                                    7],
                                   [13, 'simulator | reward | characters | hacking | simulators', 7],
                                   [14, 'naming | named | mommunes | names | reddit', 7],
                                   [15, 'feedback | untrusted | editing | trusted | monitoring', 6],
                                   [16, 'agents | village | zochi | their | gpt', 6],
                                   [17, 'llms | game | delegate | concepts | answer', 4]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': 'HhkUEA0NCgkICAgHBwcHBgYE', 'dtype': 'i1'},
                         'sizemode': 'area',
                         'sizeref': 0.01875,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': ('lknOQN7QBUEXc6k/5ivLP1dp1T8hrw' ... 'HBQJJBwEDYUac/Z6viQIDIDEGykbZA'),
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': ('j6uawGhxiMBlVgdBE0n3QCW//0AaOo' ... 'C4wIsYpsCi2A5BmJJUQLgTj8Be9rDA'),
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(0.9330164),
                                'y': np.float32(1.8194098),
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': np.float32(5.5259047),
                                'xshift': 10,
                                'y': np.float32(10.267073)}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(5.5259047),
                           'x1': np.float32(5.5259047),
                           'y0': np.float32(-6.628253),
                           'y1': np.float32(10.267073)},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(0.9330164),
                         

In [65]:
topic_model.visualize_topics(use_ctfidf=True)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'advisor | 01 | principal | alignment | sharkey', 30],
                                   [1, 'data | onionai | long | his | tokens', 25],
                                   [2, 'technical | governance | ai | policy | work', 20],
                                   [3, 'opus | claude | anthropic | it | do', 16],
                                   [4, 'material | variables | day | lunch | convenience', 13],
                                   [5, 'estimates | scorecard | hours | costs | compliance', 13],
                                   [6, 'wishes | wish | all | jobs | ai', 10],
                                   [7, 'elicitation | site | evals | results | ads', 9],
                                   [8, 'steering | vector | embeddings | spatial | vectors', 8],
                                   [9, 'advocacy | political | voting | regulations | bill', 8],
                                   [10, 'tex | font | mjxc | family | src', 8],
                                   [11, 'advocacy | donate | donating | grantmaker | soares', 7],
                                   [12, 'polysemanticity | networks | superposition | claim | solutions',
                                    7],
                                   [13, 'simulator | reward | characters | hacking | simulators', 7],
                                   [14, 'naming | named | mommunes | names | reddit', 7],
                                   [15, 'feedback | untrusted | editing | trusted | monitoring', 6],
                                   [16, 'agents | village | zochi | their | gpt', 6],
                                   [17, 'llms | game | delegate | concepts | answer', 4]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': 'HhkUEA0NCgkICAgHBwcHBgYE', 'dtype': 'i1'},
                         'sizemode': 'area',
                         'sizeref': 0.01875,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': ('EzCDQRXsgEF7yYhBnCNwQY6okUEJ/4' ... 'MiwOMRU0FDtzDAj59nQVMyY0HgeFVB'),
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': ('fIroPvjP1z3Igm8/+c6pvmLxBT//+i' ... 'NuwQvEokEjSHLBoeFCv3V+kb+C96NB'),
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(-3.4340563),
                                'y': np.float32(3.087658),
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': np.float32(8.9014225),
                                'xshift': 10,
                                'y': np.float32(23.848019)}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(8.9014225),
                           'x1': np.float32(8.9014225),
                           'y0': np.float32(-17.672703),
                           'y1': np.float32(23.848019)},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(-3.4340563),
                       

In [66]:
topic_model.visualize_heatmap()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_advisor_01_principal, 1_data_onionai_long,
                    2_technical_governance_ai, 3_opus_claude_anthropic,
                    4_material_variables_day, 5_estimates_scorecard_hours,
                    6_wishes_wish_all, 7_elicitation_site_evals,
                    8_steering_vector_embeddings, 9_advocacy_political_voting,
                    10_tex_font_mjxc, 11_advocacy_donate_donating,
                    12_polysemanticity_networks..., 13_simulator_reward_charact...,
                    14_naming_named_mommunes, 15_feedback_untrusted_editing,
                    16_agents_village_zochi, 17_llms_game_delegate],
              'xaxis': 'x',
              'y': [0_advisor_01_principal, 1_data_onionai_long,
                    2_technical_governance_ai, 3_opus_claude_anthropic,
                    4_material_variables_day, 5_estimates_scorecard_hours,
                    6_wishes_wish_all, 7_elicitation_site_evals,
                    8_steering_vector_embeddings, 9_advocacy_political_voting,
                    10_tex_font_mjxc, 11_advocacy_donate_donating,
                    12_polysemanticity_networks..., 13_simulator_reward_charact...,
                    14_naming_named_mommunes, 15_feedback_untrusted_editing,
                    16_agents_village_zochi, 17_llms_game_delegate],
              'yaxis': 'y',
              'z': {'bdata': ('AgAAAAAA8D+4sj1lJu7hP9ATa357CN' ... 'lS6JSe1D9jrBcMy1PVPwMAAAAAAPA/'),
                    'dtype': 'f8',
                    'shape': '18, 18'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})